In [86]:
import ssl
import os
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
os.environ["REQUESTS_CA_BUNDLE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")
os.environ["SSL_CERT_FILE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")

## Model Download

In [7]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org cmake
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

  Using cached cmake-3.30.1-py3-none-win_amd64.whl.metadata (6.1 kB)
Using cached cmake-3.30.1-py3-none-win_amd64.whl (35.7 MB)
Using pip 24.0 from C:\ProgramData\Anaconda3\envs\mchatbot\lib\site-packages\pip (python 3.8)
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:

  Running command pip subprocess to install build dependencies
    Using cached scikit_build-0.18.0-py3-none-any.whl.metadata (17 kB)
    Using cached cmake-3.30.1-py3-none-win_amd64.whl.metadata (6.1 kB)
    Using cached ninja-1.11.1.1-py2.py3-none-win_amd64.whl.metadata (5.4 kB)
    Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
    Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
    Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
    Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
      -------------------------

In [87]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [88]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [7]:
# code for download the model from huggingface
# model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename, local_dir=r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Learnings\GitHub\End-to-End-Medical-Chatbot-using-LLM\model")

In [89]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [90]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [91]:
## Extract the data from pdf
def load_pdf(data):
    loader =DirectoryLoader(data, glob="*.pdf", loader_cls=PyMuPDFLoader)

    documents = loader.load()
    return documents

In [92]:
extracted_data = load_pdf('data/')

In [93]:
# extracted_data

In [94]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [95]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

5779


In [96]:
## download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [97]:
embeddings = download_hugging_face_embedding()

In [98]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [99]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


## PineCone vectorDB initialisation

In [136]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [153]:
from pinecone.grpc import PineconeGRPC as Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [155]:
index_name = "medical-chatbot"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
index = pc.Index("medical-chatbot")



MaxRetryError: HTTPSConnectionPool(host='api.pinecone.io', port=443): Max retries exceeded with url: /indexes (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1135)')))

In [144]:

vector_lst = []
for i in text_chunks:
    vector_lst.append(embeddings.embed_query(i.page_content))

In [149]:
type(vector_lst[0])

list

In [152]:
index.upsert(
  vectors=[
    {"id": "A", "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]},
    {"id": "B", "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
    {"id": "C", "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
    {"id": "D", "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}
  ]
)# docsearch = pc.from_texts([t.page_content for t in text_chunks], embeddings, index_name = index_name)

PineconeException: UNKNOWN:Error received from peer  {created_time:"2024-07-30T16:52:45.8101807+00:00", grpc_status:3, grpc_message:"Vector dimension 8 does not match the dimension of the index 384"}

In [142]:
from pinecone.grpc import PineconeGRPC as Pinecone

# If interacting with Pinecone via HTTP requests, use:
# from pinecone import Pinecone

index = pc.Index("medical-chatbot")

index.upsert(
  vectors=[
    {"id": "A", "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]},
    {"id": "B", "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
    {"id": "C", "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]},
    {"id": "D", "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}
  ]
)


PineconeException: UNKNOWN:Error received from peer  {grpc_message:"Vector dimension 8 does not match the dimension of the index 384", grpc_status:3, created_time:"2024-07-30T16:42:43.2993139+00:00"}

In [101]:
## Initialise the pinecone
from langchain.vectorstores import Pinecone

In [102]:
# !pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pinecone-client


In [104]:
index_name = 'medical-chatbot'

In [105]:
import requests
import urllib3
from urllib3.exceptions import InsecureRequestWarning

urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


In [106]:
class SSLAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_context'] = self._create_unverified_context()
        return super(SSLAdapter, self).init_poolmanager(*args, **kwargs)

    def _create_unverified_context(self):
        import ssl
        context = ssl.create_default_context()
        context.check_hostname = False
        context.verify_mode = ssl.CERT_NONE
        return context


In [112]:
session = requests.Session()
session.mount('https://', SSLAdapter())
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
if index_name not in pc.list_indexes().names():
    print(index_name)

In [114]:

pc = Pinecone(api_key=PINECONE_API_KEY, host=PINE_CONE_HOST)
if index_name not in pc.list_indexes().names():
    print(index_name)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 30 Jul 2024 10:58:31 GMT', 'Content-Length': '0', 'Connection': 'keep-alive', 'server': 'envoy'})
